In [ ]:
import httplib
import urllib
import base64
import json
import ijson
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
%matplotlib inline

# you have to sign up for an API key, which has some allowances. Check the API documentation for further details:
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognizeInVideo'
_key = 'ea64a582cd2b489296e32b71f45f21c6' #Here you have to paste your primary key
_maxNumRetries = 10


In [ ]:
paramsPost = urllib.urlencode({'outputStyle' : 'perFrame', 'file':'/Volumes/data/Movies/gots7.mp4'})

headersPost = dict()
headersPost['Ocp-Apim-Subscription-Key'] = _key
headersPost['Content-Type'] = 'application/octet-stream'
jsonGet={}
headersGet = dict()
headersGet['Ocp-Apim-Subscription-Key'] = _key
paramsGet = urllib.urlencode({}) 

#Here I'm using a local file that I converted from a clip on youtube. You can also use a URL as well.
responsePost = requests.request( 'post', _url + "?" + paramsPost, \
                                data = open('/Volumes/data/Movies/gots7.mp4','rb').read(), \
                                headers = headersPost)

#A response code of 202 is what we're looking for.
print responsePost.status_code


In [ ]:
videoIDLocation = responsePost.headers['Operation-Location']
print videoIDLocation

In [ ]:
getResponse = requests.request( 'get', videoIDLocation, \
                                  data = None, headers = headersGet, params = paramsGet )

rawData = json.loads(json.loads(getResponse.text)['processingResult'])


In [ ]:
#The data that the API returns is a mix of dictionaries and lists within dictionaries.
#Because of this we have to do a little data munging to get it in a format we can work with.

emotionPerFramePerFace = {}
currFrameNum = 0

for currFragment in rawData['fragments']:
    if 'events' in currFragment:
        for currEvent in currFragment['events']:
            emotionPerFramePerFace[currFrameNum] = currEvent
            currFrameNum += 1

In [ ]:
print emotionPerFramePerFace[0]

In [ ]:
#To get our data into a format that pandas can use we create a list of dictionaries. 
gotemotions = []
for frame_no, v in emotionPerFramePerFace.copy().items():
    for i, minidict in enumerate(v):
        for k, v in minidict['scores'].items():
            minidict[k] = v
        minidict['frame'] = frame_no
        gotemotions.append(minidict)

In [ ]:
dfgot = pd.DataFrame(gotemotions)

In [ ]:
dfgot.describe()

In [1]:
# It looks like we got our counts right. Below I removed the "neutral" column.
# The neutral emotion had the highest average. I'd be interested to find out if
# the API uses "neutral" if it can't really determine what emotion the face is displaying 
# and if the neutral score decreases as they tune their algorithm.

In [3]:
emotions = dfgot[['anger', 'contempt', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']]
emotions.head()

In [ ]:
avgemotion = emotions.mean()
avg_plot = avgemotion.plot(kind='bar', legend=None, title='Avg. Emotions')
avg_plot.set_xlabel('Emotions')
avg_plot.set_ylabel('Score')